In [114]:
import requests
from bs4 import BeautifulSoup
import json 
import pandas as pd
import sqlalchemy as sql
import itertools
import numpy as np
import time

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

frq_list = [('A', '연간'), ('Q', '분기')]
finGubun_list = [('D', 'IFRS연결'), ('B', 'KIFRS별도')]

df_list = []
for i in range(0, len(company_df)):
    com = company_df.iloc[i]
    com_code = com['종목코드']
    com_name = com['종목명']
    for finGubun, frq in itertools.product(finGubun_list,  frq_list):
        total_data = []
        query={
            'cmp_cd': com_code,
            'frq': frq[0], #A 연간, Q 분기
            'finGubun': finGubun[0] #D 연결 #B 별도
            }

        url_tmpl = 'http://comp.fnguide.com/SVO2/json/data/01_06/01_A{cmp_cd}_{frq}_{finGubun}.json'
        url = url_tmpl.format(cmp_cd=query['cmp_cd'], finGubun=query['finGubun'], frq=query['frq'])
        headers={'Referer': url}
        try:
            jo = json.loads(requests.get(url, headers=headers).text)
            df = pd.DataFrame(jo['comp']).iloc[:, 3:]
        except:
            df = pd.DataFrame([])
        for i in range(1, len(df.columns)):
            dfs = df.iloc[:,i]
            date_result = dfs[0][:7]
            date_gubun = dfs[0][7:]
            if date_gubun == '(E)':
                컨센서스여부 = '컨센서스'
            else: 
                컨센서스여부 = '실적'
            Time = pd.to_datetime(date_result)
            if frq[0] == 'A':
                result_list = [com_code, com_name, frq[1], finGubun[1], Time.year, np.nan, 컨센서스여부]
            else:
                result_list = [com_code, com_name, frq[1], finGubun[1], Time.year, Time.quarter, 컨센서스여부]
            for j in dfs[1:]:
                data_value = j.replace(',', '')
                try:
                    data_value = float(data_value)
                except:
                    data_value = np.nan
                result_list.append(data_value)
            total_data.append(result_list)
        column_list = ['종목코드', '종목명', '구분', '회계', '년도', '분기', '컨센서스여부']
        if not df.empty:
            column_names = []
            for x in list(df.iloc[:,0][1:]):
                if x == '영엽이익(억원)':
                    x = '영업이익(억원)'
                column_names.append(x.replace(' ', ''))
            column_list = column_list+column_names
            df = pd.DataFrame(total_data, columns= column_list)
            df = df.loc[:, ~df.columns.duplicated(keep=False)]
            df_list.append(df)

df_total = pd.concat(df_list)
df_total.insert(loc=0, column='No', value=np.nan)

#df_total

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4),
}
for i in list(df_total.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 

df_year = df_total.loc[df_total['구분'] == '연간']
df_quarter = df_total.loc[df_total['구분'] == '분기']

No_list = []
for i in range(0, len(df_year)):
    No_list.append(i+1)
df_year['No'] = No_list

No_list = []
for i in range(0, len(df_quarter)):
    No_list.append(i+1)
df_quarter['No'] = No_list

df_year.to_sql(name="FnGuide_컨센서스_연간데이터", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `FnGuide_컨센서스_연간데이터` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `FnGuide_컨센서스_연간데이터` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `FnGuide_컨센서스_연간데이터` ADD INDEX(`종목코드`);')
    
    
df_quarter.to_sql(name="FnGuide_컨센서스_분기데이터", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `FnGuide_컨센서스_분기데이터` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `FnGuide_컨센서스_분기데이터` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `FnGuide_컨센서스_분기데이터` ADD INDEX(`종목코드`);')
    
now = time.localtime()
current_time = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

print("[" + current_time + "] " + "FnGuide_컨센서스_연간데이터, FnGuide_컨센서스_분기데이터 테이블 업데이트 되었습니다.")

,No,종목코드,종목명,구분,회계,년도,분기,컨센서스여부,매출액(억원),영업이익(억원),...,EPS(원),BPS(원),DPS(원),PER(배),PBR(배),보험료수익(억원),순영업수익(억원),이자수익(억원),영업손익(억원),영업수익(억원)
0,1,000020,동화약품,연간,IFRS연결,2017.0,NaN,실적,2589.0,110.0,...,1683.0,10721.0,150.0,5.79,0.91,NaN,NaN,NaN,NaN,NaN
1,2,000020,동화약품,연간,IFRS연결,2018.0,NaN,실적,3066.0,112.0,...,360.0,10727.0,120.0,25.24,0.85,NaN,NaN,NaN,NaN,NaN
2,3,000020,동화약품,연간,IFRS연결,2019.0,NaN,실적,3072.0,99.0,...,337.0,10862.0,120.0,24.69,0.77,NaN,NaN,NaN,NaN,NaN
3,4,000020,동화약품,연간,IFRS연결,2020.0,NaN,컨센서스,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,000020,동화약품,연간,IFRS연결,2021.0,NaN,컨센서스,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,57212,122830,원포유,분기,KIFRS별도,2020.0,1.0,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57213,122830,원포유,분기,KIFRS별도,2020.0,2.0,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,57214,122830,원포유,분기,KIFRS별도,2020.0,3.0,컨센서스,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57215,122830,원포유,분기,KIFRS별도,2020.0,4.0,컨센서스,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
